In [1]:
#Academy of Py
#Dependencies

import pandas as pd
import os

In [2]:
#Load Files

csvpath = os.path.join('Resources', 'schools_complete.csv')
csvpath2 = os.path.join('Resources', 'students_complete.csv')

schools_df = pd.read_csv(csvpath)
students_df = pd.read_csv(csvpath2)


In [3]:
#Show columns and snapshots of 1st data file

schools_df = schools_df.rename(columns={"name": "School Name"})

schools_df.head()

,School ID,School Name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [12]:
#Show columns and snapshots of 2nd data file

students_df.head()

#Calculate the total school data (total budget, total size, number of schools)

total_schools = schools_df["School Name"].count()
total_students = schools_df["size"].sum()
total_budget = schools_df["budget"].sum()

total_schools
total_students
total_budget

24649428

In [24]:
#Show District Summary showing school name (y), school type (y), total students (y), total school budget (y), 
#per student budget, average math score (y), average reading score (y), %passing math, %passing reading, 
#overall passing rate (average of prior 2)

#create a groupby data frame using the school average

groupbyavg_school = students_df.groupby("school").mean()
groupbyavg_school.head()

groupbyavg_school = groupbyavg_school.reset_index()

groupbyavg_school = groupbyavg_school.rename(columns={"school":"School Name",
                                                     "reading_score":"Avg Reading Score",
                                                     "math_score":"Avg Math Score"})
#Calculate Average Reading and Math Scores

ave_read_score = groupbyavg_school["Avg Reading Score"].mean()
ave_math_score = groupbyavg_school["Avg Math Score"].mean()

groupbyavg_school.head()

,School Name,Student ID,Avg Reading Score,Avg Math Score
0,Bailey High School,20358.5,81.033963,77.048432
1,Cabrera High School,16941.5,83.975780,83.061895
2,Figueroa High School,4391.0,81.158020,76.711767
3,Ford High School,36165.0,80.746258,77.102592
4,Griffin High School,12995.5,83.816757,83.351499


In [22]:
#Calculate the passing reading and passing math (assuming passing is 70% or more)

students_df.head()

passing_students_read = students_df.loc[students_df["reading_score"]>=70]

passing_students_math = students_df.loc[students_df["math_score"]>=70]

#create a data frame with value counts of passing in reading

passing_read_school = passing_students_read["school"].value_counts()
passing_read_school = passing_read_school.reset_index()
passing_read_school = passing_read_school.rename(columns={"index":"School Name", "school":"Passing Read Total"})

#Calculate the Passing Reading Total and Percentage
passing_read_total = passing_read_school["Passing Read Total"].sum()
passing_read_percent = passing_read_total/total_students*100

passing_read_school.head()

#create a data frame with value counts of passing in reading

passing_math_school = passing_students_math["school"].value_counts()
passing_math_school = passing_math_school.reset_index()
passing_math_school = passing_math_school.rename(columns={"index":"School Name", "school":"Passing Math Total"})
passing_math_school.head()

#Calculate the Passing Math Total and Percentage
passing_math_total = passing_math_school["Passing Math Total"].sum()
passing_math_percent = passing_math_total/total_students*100

#Calculate the combined Passing Reading and Math Percentage
average_passing = (passing_read_percent+passing_math_percent)/2

In [192]:
#Create a new data frame with Overall School Summary

school_summary = pd.DataFrame({"Total Schools":[total_schools], "Total Students":[total_students],
                              "Total Budget":[total_budget], "Average Read Score":[ave_read_score],
                              "Average Math Score":[ave_math_score], "% Passing Read":[passing_read_percent],
                              "% Passing Math":[passing_math_percent], "% Overall Passing":[average_passing]})

school_summary

#Rearrange the data
school_summary = school_summary[["Total Schools","Total Students", "Total Budget", "Average Read Score",
                                "Average Math Score", "% Passing Read", "% Passing Math",
                                "% Overall Passing"]]

school_summary

#Finalize configuration

school_summary["Total Budget"] = school_summary["Total Budget"].map("$ {:,.2f}".format)
school_summary["% Passing Read"] = school_summary["% Passing Read"].map("{:,.2f} %".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:,.2f} %".format)
school_summary["% Overall Passing"] = school_summary["% Overall Passing"].map("{:,.2f} %".format)

school_summary


,Total Schools,Total Students,Total Budget,Average Read Score,Average Math Score,% Passing Read,% Passing Math,% Overall Passing
0,15,39170,"$ 24,649,428.00",82.529188,80.43253,85.81 %,74.98 %,80.39 %


In [193]:
# Per School High Level Summary
#Merge all pertinent data frames using School Name as the common factor

merge_schoolA_df = pd.merge(groupbyavg_school, schools_df, on="School Name")
merge_schoolB_df = pd.merge(passing_read_school, passing_math_school, on="School Name")
merge_school_df = pd.merge(merge_schoolA_df, merge_schoolB_df, on="School Name")

merge_school_df = merge_school_df.rename(columns={"size":"Total Students"})

#Calculate the other related data needed and create new columns for them

merge_school_df["Passing Read %"] = merge_school_df["Passing Read Total"]/merge_school_df["Total Students"]*100
merge_school_df["Passing Math %"] = merge_school_df["Passing Math Total"]/merge_school_df["Total Students"]*100
merge_school_df["Budget per Student"] = merge_school_df["budget"]/merge_school_df["Total Students"]
merge_school_df["% Overall Passing"] = (merge_school_df["Passing Read %"]+merge_school_df["Passing Math %"])/2

merge_school_df.head()


,School Name,Student ID,Avg Reading Score,Avg Math Score,School ID,type,Total Students,budget,Passing Read Total,Passing Math Total,Passing Read %,Passing Math %,Budget per Student,% Overall Passing
0,Bailey High School,20358.5,81.033963,77.048432,7,District,4976,3124928,4077,3318,81.933280,66.680064,628.0,74.306672
1,Cabrera High School,16941.5,83.975780,83.061895,6,Charter,1858,1081356,1803,1749,97.039828,94.133477,582.0,95.586652
2,Figueroa High School,4391.0,81.158020,76.711767,1,District,2949,1884411,2381,1946,80.739234,65.988471,639.0,73.363852
3,Ford High School,36165.0,80.746258,77.102592,13,District,2739,1763916,2172,1871,79.299014,68.309602,644.0,73.804308
4,Griffin High School,12995.5,83.816757,83.351499,4,Charter,1468,917500,1426,1371,97.138965,93.392371,625.0,95.265668


In [213]:
#Rearrange the data frame

revised_school_df = merge_school_df[["School Name", "type", "Total Students", "budget", "Budget per Student",
                                    "Avg Reading Score", "Avg Math Score", "Passing Read %", "Passing Math %",
                                    "% Overall Passing"]]

revised_school_df = revised_school_df.rename(columns={"type":"School Type", "budget":"School Budget"})

revised_school_df.head()

,School Name,School Type,Total Students,School Budget,Budget per Student,Avg Reading Score,Avg Math Score,Passing Read %,Passing Math %,% Overall Passing
0,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.933280,66.680064,74.306672
1,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
2,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
3,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,79.299014,68.309602,73.804308
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,95.265668


In [214]:
#Clean-up the Formatting

revised_school_df["School Budget"] = revised_school_df["School Budget"].map("$ {:,.2f}".format)
revised_school_df["Budget per Student"] = revised_school_df["Budget per Student"].map("$ {:,.2f}".format)
revised_school_df["Avg Reading Score"] = revised_school_df["Avg Reading Score"].map("{:,.2f}".format)
revised_school_df["Avg Math Score"] = revised_school_df["Avg Math Score"].map("{:,.2f}".format)
revised_school_df["Passing Read %"] = revised_school_df["Passing Read %"].map("{:,.2f}%".format)
revised_school_df["Passing Math %"] = revised_school_df["Passing Math %"].map("{:,.2f} %".format)
revised_school_df["% Overall Passing"] = revised_school_df["% Overall Passing"].map("{:,.2f} %".format)

revised_school_df.head()

,School Name,School Type,Total Students,School Budget,Budget per Student,Avg Reading Score,Avg Math Score,Passing Read %,Passing Math %,% Overall Passing
0,Bailey High School,District,4976,"$ 3,124,928.00",$ 628.00,81.03,77.05,81.93%,66.68 %,74.31 %
1,Cabrera High School,Charter,1858,"$ 1,081,356.00",$ 582.00,83.98,83.06,97.04%,94.13 %,95.59 %
2,Figueroa High School,District,2949,"$ 1,884,411.00",$ 639.00,81.16,76.71,80.74%,65.99 %,73.36 %
3,Ford High School,District,2739,"$ 1,763,916.00",$ 644.00,80.75,77.10,79.30%,68.31 %,73.80 %
4,Griffin High School,Charter,1468,"$ 917,500.00",$ 625.00,83.82,83.35,97.14%,93.39 %,95.27 %


In [215]:
#Create Final Summary Data Frame usinng School Name as the Index

final_school_df = revised_school_df.set_index("School Name")

final_school_df.head()

,School Type,Total Students,School Budget,Budget per Student,Avg Reading Score,Avg Math Score,Passing Read %,Passing Math %,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$ 3,124,928.00",$ 628.00,81.03,77.05,81.93%,66.68 %,74.31 %
Cabrera High School,Charter,1858,"$ 1,081,356.00",$ 582.00,83.98,83.06,97.04%,94.13 %,95.59 %
Figueroa High School,District,2949,"$ 1,884,411.00",$ 639.00,81.16,76.71,80.74%,65.99 %,73.36 %
Ford High School,District,2739,"$ 1,763,916.00",$ 644.00,80.75,77.10,79.30%,68.31 %,73.80 %
Griffin High School,Charter,1468,"$ 917,500.00",$ 625.00,83.82,83.35,97.14%,93.39 %,95.27 %


In [216]:
# Top and Bottom Performing Schools

#Top Schools based on %Overall Passing

top_schools = final_school_df.sort_values("% Overall Passing", ascending=False)

top_schools.head(5)

,School Type,Total Students,School Budget,Budget per Student,Avg Reading Score,Avg Math Score,Passing Read %,Passing Math %,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$ 1,081,356.00",$ 582.00,83.98,83.06,97.04%,94.13 %,95.59 %
Thomas High School,Charter,1635,"$ 1,043,130.00",$ 638.00,83.85,83.42,97.31%,93.27 %,95.29 %
Griffin High School,Charter,1468,"$ 917,500.00",$ 625.00,83.82,83.35,97.14%,93.39 %,95.27 %
Pena High School,Charter,962,"$ 585,858.00",$ 609.00,84.04,83.84,95.95%,94.59 %,95.27 %
Wilson High School,Charter,2283,"$ 1,319,574.00",$ 578.00,83.99,83.27,96.54%,93.87 %,95.20 %


In [217]:
#Worst Schools based on %Overall Passing

worst_schools = final_school_df.sort_values("% Overall Passing", ascending=True)

worst_schools.head(5)

,School Type,Total Students,School Budget,Budget per Student,Avg Reading Score,Avg Math Score,Passing Read %,Passing Math %,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$ 2,547,363.00",$ 637.00,80.74,76.84,80.22%,66.37 %,73.29 %
Figueroa High School,District,2949,"$ 1,884,411.00",$ 639.00,81.16,76.71,80.74%,65.99 %,73.36 %
Huang High School,District,2917,"$ 1,910,635.00",$ 655.00,81.18,76.63,81.32%,65.68 %,73.50 %
Johnson High School,District,4761,"$ 3,094,650.00",$ 650.00,80.97,77.07,81.22%,66.06 %,73.64 %
Ford High School,District,2739,"$ 1,763,916.00",$ 644.00,80.75,77.10,79.30%,68.31 %,73.80 %


In [67]:
#Math and Read Scores by Grade

students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [110]:
#Create a data frame by grade using the loc and groupby method

students_df = students_df.rename(columns={"school":"School Name"})

grade9_df = students_df.loc[students_df["grade"]=="9th"]
grade10_df = students_df.loc[students_df["grade"]=="10th"]
grade11_df = students_df.loc[students_df["grade"]=="11th"]
grade12_df = students_df.loc[students_df["grade"]=="12th"]

grade9rev_df = grade9_df.groupby("School Name").mean()
grade10rev_df = grade10_df.groupby("School Name").mean()
grade11rev_df = grade11_df.groupby("School Name").mean()
grade12rev_df = grade12_df.groupby("School Name").mean()

grade9rev_df = grade9rev_df.rename(columns={"reading_score":"9th Grade Reading Avg", 
                                            "math_score":"9th Grade Math Avg"})

grade10rev_df = grade10rev_df.rename(columns={"reading_score":"10th Grade Reading Avg", 
                                            "math_score":"10th Grade Math Avg"})
grade11rev_df = grade11rev_df.rename(columns={"reading_score":"11th Grade Reading Avg", 
                                            "math_score":"11th Grade Math Avg"})
grade12rev_df = grade12rev_df.rename(columns={"reading_score":"12th Grade Reading Avg", 
                                            "math_score":"12th Grade Math Avg"})

grade12rev_df.head()

,Student ID,12th Grade Reading Avg,12th Grade Math Avg
School Name,,,
Bailey High School,20386.724708,80.912451,76.492218
Cabrera High School,16924.570681,84.287958,83.277487
Figueroa High School,4414.922705,81.384863,77.151369
Ford High School,36153.562152,80.662338,76.179963
Griffin High School,12977.143836,84.013699,83.356164


In [111]:
#Merge new Data Frames

grade9rev_df = grade9rev_df.reset_index()
grade10rev_df = grade10rev_df.reset_index()
grade11rev_df = grade11rev_df.reset_index()
grade12rev_df = grade12rev_df.reset_index()

schools_bygradeA = pd.merge(grade9rev_df, grade10rev_df, on= "School Name")
schools_bygradeB = pd.merge(grade11rev_df, grade12rev_df, on= "School Name")
schools_bygrade = pd.merge(schools_bygradeA, schools_bygradeB, on= "School Name")

In [119]:
schools_bygrade.head()

schools_bygrade_math = schools_bygrade[["School Name", "9th Grade Math Avg", "10th Grade Math Avg", 
                                    "11th Grade Math Avg", "12th Grade Math Avg"]]

schools_bygrade_read = schools_bygrade[["School Name", "9th Grade Reading Avg", "10th Grade Reading Avg", 
                                    "11th Grade Reading Avg", "12th Grade Reading Avg"]]

In [124]:
#Average Math Score by School and Grade

schools_bygrade_math = schools_bygrade_math.set_index("School Name")
schools_bygrade_math.head()

,9th Grade Math Avg,10th Grade Math Avg,11th Grade Math Avg,12th Grade Math Avg
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


In [125]:
#Average Read Score by School and Grade

schools_bygrade_read = schools_bygrade_read.set_index("School Name")
schools_bygrade_read.head()

,9th Grade Reading Avg,10th Grade Reading Avg,11th Grade Reading Avg,12th Grade Reading Avg
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


In [142]:
#Scores by School Spending
#Use merge_school_df as the base file

merge_school_df.head()

,School Name,Student ID,Avg Reading Score,Avg Math Score,School ID,type,Total Students,budget,Passing Read Total,Passing Math Total,Passing Read %,Passing Math %,Budget per Student,% Overall Passing
0,Bailey High School,20358.5,81.033963,77.048432,7,District,4976,3124928,4077,3318,81.933280,66.680064,628.0,74.306672
1,Cabrera High School,16941.5,83.975780,83.061895,6,Charter,1858,1081356,1803,1749,97.039828,94.133477,582.0,95.586652
2,Figueroa High School,4391.0,81.158020,76.711767,1,District,2949,1884411,2381,1946,80.739234,65.988471,639.0,73.363852
3,Ford High School,36165.0,80.746258,77.102592,13,District,2739,1763916,2172,1871,79.299014,68.309602,644.0,73.804308
4,Griffin High School,12995.5,83.816757,83.351499,4,Charter,1468,917500,1426,1371,97.138965,93.392371,625.0,95.265668


In [143]:
print(merge_school_df["Budget per Student"].min())
print(merge_school_df["Budget per Student"].max())

578.0
655.0


In [195]:
#Set-up the binning and names

bins = [0, 585, 610, 630, 655]

group_names = ["< $585", "$585 - 610", "$610 - 630", "$630 - 655"]

revised_school2_df = merge_school_df

revised_school2_df["Spending Ranges per Student"] = pd.cut(revised_school2_df["Budget per Student"], bins, 
                                                           labels=group_names)

In [196]:
revised_school2_df.head()

,School Name,Student ID,Avg Reading Score,Avg Math Score,School ID,type,Total Students,budget,Passing Read Total,Passing Math Total,Passing Read %,Passing Math %,Budget per Student,% Overall Passing,Spending Ranges per Student
0,Bailey High School,20358.5,81.033963,77.048432,7,District,4976,3124928,4077,3318,81.933280,66.680064,628.0,74.306672,$610 - 630
1,Cabrera High School,16941.5,83.975780,83.061895,6,Charter,1858,1081356,1803,1749,97.039828,94.133477,582.0,95.586652,< $585
2,Figueroa High School,4391.0,81.158020,76.711767,1,District,2949,1884411,2381,1946,80.739234,65.988471,639.0,73.363852,$630 - 655
3,Ford High School,36165.0,80.746258,77.102592,13,District,2739,1763916,2172,1871,79.299014,68.309602,644.0,73.804308,$630 - 655
4,Griffin High School,12995.5,83.816757,83.351499,4,Charter,1468,917500,1426,1371,97.138965,93.392371,625.0,95.265668,$610 - 630


In [164]:
#group the dataframe by Spending Ranges

#Obtain the budget 
groupbyavg_budget = revised_school2_df.groupby("Spending Ranges per Student").mean()
groupbysum_budget = revised_school2_df.groupby("Spending Ranges per Student").sum()

#groupbyavg_budgetrev = groupbyavg_budget.reset_index()
groupbyavg_budgetrev = groupbyavg_budget[["Avg Reading Score", "Avg Math Score"]]

In [165]:
groupbyavg_budgetrev

groupbyavg_budgetrev = groupbyavg_budgetrev.reset_index()
groupbyavg_budgetrev

,Spending Ranges per Student,Avg Reading Score,Avg Math Score
0,< $585,83.933814,83.455399
1,$585 - 610,83.885211,83.599686
2,$610 - 630,82.425360,80.199966
3,$630 - 655,81.368774,77.866721


In [167]:
#Obtain the percentage data

groupbysum_budgetrev = groupbysum_budget[["Total Students", "Passing Read Total", "Passing Math Total"]]

groupbysum_budgetrev = groupbysum_budgetrev.reset_index()

groupbysum_budgetrev["% Passing Read"] = groupbysum_budgetrev["Passing Read Total"]/groupbysum_budgetrev["Total Students"]*100
groupbysum_budgetrev["% Passing Math"] = groupbysum_budgetrev["Passing Math Total"]/groupbysum_budgetrev["Total Students"]*100
groupbysum_budgetrev["% Percent Overall Rate"] = (groupbysum_budgetrev["% Passing Read"]+
                                                  groupbysum_budgetrev["% Passing Math"])/2

groupbysum_budgetrev2 = groupbysum_budgetrev[["Spending Ranges per Student", "% Passing Read", "% Passing Math",
                                             "% Percent Overall Rate"]]
groupbysum_budgetrev2

,Spending Ranges per Student,% Passing Read,% Passing Math,% Percent Overall Rate
0,< $585,96.686558,93.702889,95.194724
1,$585 - 610,95.886889,94.124128,95.005509
2,$610 - 630,85.397269,72.765363,79.081316
3,$630 - 655,81.823567,68.335096,75.079331


In [223]:
#Merge the two budget data together using Spending Ranges per Student

merge_school_budget_df = pd.merge(groupbyavg_budgetrev, groupbysum_budgetrev2, on="Spending Ranges per Student")
final_school_budget = merge_school_budget_df
final_school_budget

,Spending Ranges per Student,Avg Reading Score,Avg Math Score,% Passing Read,% Passing Math,% Percent Overall Rate
0,< $585,83.933814,83.455399,96.686558,93.702889,95.194724
1,$585 - 610,83.885211,83.599686,95.886889,94.124128,95.005509
2,$610 - 630,82.425360,80.199966,85.397269,72.765363,79.081316
3,$630 - 655,81.368774,77.866721,81.823567,68.335096,75.079331


In [224]:
final_school_budget = final_school_budget.set_index("Spending Ranges per Student")

In [225]:
#Final Summary by School Budget per Student

#Clean-up the Formatting

final_school_budget["Avg Reading Score"] = final_school_budget["Avg Reading Score"].map("{:,.2f}".format)
final_school_budget["Avg Math Score"] = final_school_budget["Avg Math Score"].map("{:,.2f}".format)
final_school_budget["% Passing Read"] = final_school_budget["% Passing Read"].map("{:,.2f}%".format)
final_school_budget["% Passing Math"] = final_school_budget["% Passing Math"].map("{:,.2f} %".format)
final_school_budget["% Percent Overall Rate"] = final_school_budget["% Percent Overall Rate"].map("{:,.2f} %".format)

final_school_budget

,Avg Reading Score,Avg Math Score,% Passing Read,% Passing Math,% Percent Overall Rate
Spending Ranges per Student,,,,,
< $585,83.93,83.46,96.69%,93.70 %,95.19 %
$585 - 610,83.89,83.60,95.89%,94.12 %,95.01 %
$610 - 630,82.43,80.20,85.40%,72.77 %,79.08 %
$630 - 655,81.37,77.87,81.82%,68.34 %,75.08 %


In [173]:
#Scores by School Size
#Use merge_school_df as the base file

merge_school_df.head()

,School Name,Student ID,Avg Reading Score,Avg Math Score,School ID,type,Total Students,budget,Passing Read Total,Passing Math Total,Passing Read %,Passing Math %,Budget per Student,% Overall Passing,Spending Ranges per Student
0,Bailey High School,20358.5,81.033963,77.048432,7,District,4976,3124928,4077,3318,81.933280,66.680064,628.0,74.306672,$610 - 630
1,Cabrera High School,16941.5,83.975780,83.061895,6,Charter,1858,1081356,1803,1749,97.039828,94.133477,582.0,95.586652,< $585
2,Figueroa High School,4391.0,81.158020,76.711767,1,District,2949,1884411,2381,1946,80.739234,65.988471,639.0,73.363852,$630 - 655
3,Ford High School,36165.0,80.746258,77.102592,13,District,2739,1763916,2172,1871,79.299014,68.309602,644.0,73.804308,$630 - 655
4,Griffin High School,12995.5,83.816757,83.351499,4,Charter,1468,917500,1426,1371,97.138965,93.392371,625.0,95.265668,$610 - 630


In [174]:
print(merge_school_df["Total Students"].min())
print(merge_school_df["Total Students"].max())

427
4976


In [178]:
#Set-up the binning and names

bins_size = [0, 1000, 3000, 5000]

group_names_size = ["Small", "Medium", "Large"]

revised_school3_df = merge_school_df

revised_school3_df["School Size"] = pd.cut(revised_school3_df["Total Students"], bins_size, 
                                                           labels=group_names_size)
#revised_school3_df.head()

In [183]:
#group the dataframe by School Size

#Obtain the size
groupbyavg_size = revised_school3_df.groupby("School Size").mean()
groupbysum_size = revised_school3_df.groupby("School Size").sum()

groupbyavg_sizerev = groupbyavg_size[["Avg Reading Score", "Avg Math Score"]]

groupbyavg_sizerev

groupbyavg_sizerev = groupbyavg_sizerev.reset_index()
groupbyavg_sizerev

,School Size,Avg Reading Score,Avg Math Score
0,Small,83.929843,83.821598
1,Medium,82.933187,81.176821
2,Large,80.919864,77.063340


In [188]:
#Obtain the percentage data

groupbysum_sizerev = groupbysum_size[["Total Students","Passing Read Total", "Passing Math Total"]]

groupbysum_sizerev = groupbysum_sizerev.reset_index()

groupbysum_sizerev["% Passing Read"] = groupbysum_sizerev["Passing Read Total"]/groupbysum_sizerev["Total Students"]*100
groupbysum_sizerev["% Passing Math"] = groupbysum_sizerev["Passing Math Total"]/groupbysum_sizerev["Total Students"]*100
groupbysum_sizerev["% Percent Overall Rate"] = (groupbysum_sizerev["% Passing Read"]+
                                                 groupbysum_sizerev["% Passing Math"])/2

groupbysum_sizerev2 = groupbysum_sizerev[["School Size", "% Passing Read", "% Passing Math",
                                             "% Percent Overall Rate"]]
groupbysum_sizerev2

,School Size,% Passing Read,% Passing Math,% Percent Overall Rate
0,Small,96.040317,93.952484,94.996400
1,Medium,89.520866,81.679547,85.600206
2,Large,81.106091,66.468891,73.787491


In [226]:
#Merge the two budget data together using Spending Ranges per Student

merge_school_size_df = pd.merge(groupbyavg_sizerev, groupbysum_sizerev2, on="School Size")
final_school_size = merge_school_size_df
final_school_size

,School Size,Avg Reading Score,Avg Math Score,% Passing Read,% Passing Math,% Percent Overall Rate
0,Small,83.929843,83.821598,96.040317,93.952484,94.996400
1,Medium,82.933187,81.176821,89.520866,81.679547,85.600206
2,Large,80.919864,77.063340,81.106091,66.468891,73.787491


In [227]:
final_school_size = final_school_size.set_index("School Size")

#Final Summary by School Budget per Student

#Clean-up the Formatting

final_school_size["Avg Reading Score"] = final_school_size["Avg Reading Score"].map("{:,.2f}".format)
final_school_size["Avg Math Score"] = final_school_size["Avg Math Score"].map("{:,.2f}".format)
final_school_size["% Passing Read"] = final_school_size["% Passing Read"].map("{:,.2f}%".format)
final_school_size["% Passing Math"] = final_school_size["% Passing Math"].map("{:,.2f} %".format)
final_school_size["% Percent Overall Rate"] = final_school_size["% Percent Overall Rate"].map("{:,.2f} %".format)


final_school_size

,Avg Reading Score,Avg Math Score,% Passing Read,% Passing Math,% Percent Overall Rate
School Size,,,,,
Small,83.93,83.82,96.04%,93.95 %,95.00 %
Medium,82.93,81.18,89.52%,81.68 %,85.60 %
Large,80.92,77.06,81.11%,66.47 %,73.79 %


In [194]:
#Scores by School Type
#Use merge_school_df as the base file

merge_school_df.head()

,School Name,Student ID,Avg Reading Score,Avg Math Score,School ID,type,Total Students,budget,Passing Read Total,Passing Math Total,Passing Read %,Passing Math %,Budget per Student,% Overall Passing
0,Bailey High School,20358.5,81.033963,77.048432,7,District,4976,3124928,4077,3318,81.933280,66.680064,628.0,74.306672
1,Cabrera High School,16941.5,83.975780,83.061895,6,Charter,1858,1081356,1803,1749,97.039828,94.133477,582.0,95.586652
2,Figueroa High School,4391.0,81.158020,76.711767,1,District,2949,1884411,2381,1946,80.739234,65.988471,639.0,73.363852
3,Ford High School,36165.0,80.746258,77.102592,13,District,2739,1763916,2172,1871,79.299014,68.309602,644.0,73.804308
4,Griffin High School,12995.5,83.816757,83.351499,4,Charter,1468,917500,1426,1371,97.138965,93.392371,625.0,95.265668


In [198]:
revised_school4_df = merge_school_df

#group the dataframe by School Type

#Obtain the type
groupbyavg_type = revised_school4_df.groupby("type").mean()
groupbysum_type = revised_school4_df.groupby("type").sum()

groupbyavg_typerev = groupbyavg_type[["Avg Reading Score", "Avg Math Score"]]

groupbyavg_typerev = groupbyavg_typerev.reset_index()
groupbyavg_typerev

,type,Avg Reading Score,Avg Math Score
0,Charter,83.896421,83.473852
1,District,80.966636,76.956733


In [200]:
#Obtain the percentage data

groupbysum_typerev = groupbysum_type[["Total Students","Passing Read Total", "Passing Math Total"]]

groupbysum_typerev = groupbysum_typerev.reset_index()

groupbysum_typerev["% Passing Read"] = groupbysum_typerev["Passing Read Total"]/groupbysum_typerev["Total Students"]*100
groupbysum_typerev["% Passing Math"] = groupbysum_typerev["Passing Math Total"]/groupbysum_typerev["Total Students"]*100
groupbysum_typerev["% Percent Overall Rate"] = (groupbysum_typerev["% Passing Read"]+
                                                 groupbysum_typerev["% Passing Math"])/2

groupbysum_typerev2 = groupbysum_typerev[["type", "% Passing Read", "% Passing Math",
                                             "% Percent Overall Rate"]]
groupbysum_typerev2

,type,% Passing Read,% Passing Math,% Percent Overall Rate
0,Charter,96.645891,93.701821,95.173856
1,District,80.905249,66.518387,73.711818


In [228]:
#Merge the two budget data together using Type

merge_school_type_df = pd.merge(groupbyavg_typerev, groupbysum_typerev2, on="type")
final_school_type = merge_school_type_df
final_school_type

,type,Avg Reading Score,Avg Math Score,% Passing Read,% Passing Math,% Percent Overall Rate
0,Charter,83.896421,83.473852,96.645891,93.701821,95.173856
1,District,80.966636,76.956733,80.905249,66.518387,73.711818


In [229]:
final_school_type = final_school_type.rename(columns={"type":"School Type"})

final_school_type
final_school_type = final_school_type.set_index("School Type")

#Clean-up the Formatting

final_school_type["Avg Reading Score"] = final_school_type["Avg Reading Score"].map("{:,.2f}".format)
final_school_type["Avg Math Score"] = final_school_type["Avg Math Score"].map("{:,.2f}".format)
final_school_type["% Passing Read"] = final_school_type["% Passing Read"].map("{:,.2f}%".format)
final_school_type["% Passing Math"] = final_school_type["% Passing Math"].map("{:,.2f} %".format)
final_school_type["% Percent Overall Rate"] = final_school_type["% Percent Overall Rate"].map("{:,.2f} %".format)

#Final Summary by School Budget per Student

final_school_type

,Avg Reading Score,Avg Math Score,% Passing Read,% Passing Math,% Percent Overall Rate
School Type,,,,,
Charter,83.90,83.47,96.65%,93.70 %,95.17 %
District,80.97,76.96,80.91%,66.52 %,73.71 %
